In [9]:
import numpy as np
import pandas as pd

from imblearn.over_sampling import SMOTE

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import xgboost as xgb
from sklearn.model_selection import GridSearchCV


df = pd.read_csv("heart_disease_health_indicators_BRFSS2015.csv")
df.head()

,HeartDiseaseorAttack,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,Diabetes,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253680 entries, 0 to 253679
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   HeartDiseaseorAttack  253680 non-null  float64
 1   HighBP                253680 non-null  float64
 2   HighChol              253680 non-null  float64
 3   CholCheck             253680 non-null  float64
 4   BMI                   253680 non-null  float64
 5   Smoker                253680 non-null  float64
 6   Stroke                253680 non-null  float64
 7   Diabetes              253680 non-null  float64
 8   PhysActivity          253680 non-null  float64
 9   Fruits                253680 non-null  float64
 10  Veggies               253680 non-null  float64
 11  HvyAlcoholConsump     253680 non-null  float64
 12  AnyHealthcare         253680 non-null  float64
 13  NoDocbcCost           253680 non-null  float64
 14  GenHlth               253680 non-null  float64
 15  

In [11]:
df = df.drop(columns = ['GenHlth', 'MentHlth','PhysHlth','Income','Education','NoDocbcCost'])
df.columns

Index(['HeartDiseaseorAttack', 'HighBP', 'HighChol', 'CholCheck', 'BMI',
       'Smoker', 'Stroke', 'Diabetes', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'DiffWalk', 'Sex', 'Age'],
      dtype='object')

In [12]:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253680 entries, 0 to 253679
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   HeartDiseaseorAttack  253680 non-null  float64
 1   HighBP                253680 non-null  float64
 2   HighChol              253680 non-null  float64
 3   CholCheck             253680 non-null  float64
 4   BMI                   253680 non-null  float64
 5   Smoker                253680 non-null  float64
 6   Stroke                253680 non-null  float64
 7   Diabetes              253680 non-null  float64
 8   PhysActivity          253680 non-null  float64
 9   Fruits                253680 non-null  float64
 10  Veggies               253680 non-null  float64
 11  HvyAlcoholConsump     253680 non-null  float64
 12  AnyHealthcare         253680 non-null  float64
 13  DiffWalk              253680 non-null  float64
 14  Sex                   253680 non-null  float64
 15  

In [13]:
df['HeartDiseaseorAttack'].value_counts()
# shows inbalance in data

HeartDiseaseorAttack
0.0    229787
1.0     23893
Name: count, dtype: int64

In [14]:

for column in df.columns:
    value_counts = df[column].value_counts()
    print(f"Value counts for column '{column}':\n{value_counts}\n")

Value counts for column 'HeartDiseaseorAttack':
HeartDiseaseorAttack
0.0    229787
1.0     23893
Name: count, dtype: int64

Value counts for column 'HighBP':
HighBP
0.0    144851
1.0    108829
Name: count, dtype: int64

Value counts for column 'HighChol':
HighChol
0.0    146089
1.0    107591
Name: count, dtype: int64

Value counts for column 'CholCheck':
CholCheck
1.0    244210
0.0      9470
Name: count, dtype: int64

Value counts for column 'BMI':
BMI
27.0    24606
26.0    20562
24.0    19550
25.0    17146
28.0    16545
        ...  
85.0        1
91.0        1
86.0        1
90.0        1
78.0        1
Name: count, Length: 84, dtype: int64

Value counts for column 'Smoker':
Smoker
0.0    141257
1.0    112423
Name: count, dtype: int64

Value counts for column 'Stroke':
Stroke
0.0    243388
1.0     10292
Name: count, dtype: int64

Value counts for column 'Diabetes':
Diabetes
0.0    213703
2.0     35346
1.0      4631
Name: count, dtype: int64

Value counts for column 'PhysActivity':
Phys

In [15]:
y = df['HeartDiseaseorAttack'].values
X = df.drop(columns='HeartDiseaseorAttack', axis=1).values

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### SMOTE for imbalanced data

In [17]:
# creating a SMOTE object
smote = SMOTE(random_state=42)


In [18]:
# applying SMOTE to the training data
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

#### Creating and training  the XGBoost model

In [19]:
#creating the xgboost model
xgb_model = xgb.XGBClassifier( random_state=42)

In [20]:
#training the model
xgb_model.fit(X_train_resampled, y_train_resampled)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [21]:
# Save the XGBoost model to a file
xgb_model.save_model('./Models/xgboost_model.model')

/home/arnold/anaconda3/envs/ml/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [08:34:08] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [22]:
#Prediction of the model

y_pred_xgb = xgb_model.predict(X_test)

In [23]:
# Evaluate the model
print(confusion_matrix(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb))
print("Accuracy:", accuracy_score(y_test, y_pred_xgb))

[[42599  3369]
 [ 3166  1602]]
              precision    recall  f1-score   support

         0.0       0.93      0.93      0.93     45968
         1.0       0.32      0.34      0.33      4768

    accuracy                           0.87     50736
   macro avg       0.63      0.63      0.63     50736
weighted avg       0.87      0.87      0.87     50736

Accuracy: 0.8711959949542731


### The model is performing better than the random forest model, will use this model for the web app but first will try to improve the model using grid search 

In [24]:
# Enabling GPU support
params = {
    'tree_method': 'gpu_hist', #use GPU for training
    'predictor': 'gpu_predictor' # use GPU for inference

}

In [25]:
# Define the grid of hyperparameters to search

param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [100, 200, 300],
    'subsample': [0.8, 0.9, 1],
    'colasample_bytree': [0.8, 0.9, 1]
}


In [26]:
# creating the GridSearchCV object

grid_search = GridSearchCV(xgb.XGBClassifier(**params , random_state=42),param_grid, cv=3, scoring='f1', verbose=1, n_jobs=10)

In [27]:
# Grid Search with resampled data
grid_search.fit(X_train_resampled, y_train_resampled)


Fitting 3 folds for each of 243 candidates, totalling 729 fits


/home/arnold/anaconda3/envs/ml/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [08:35:20] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/arnold/anaconda3/envs/ml/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [08:35:20] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/arnold/anaconda3/envs/ml/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [08:35:20] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_m

GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     predictor='gpu_predictor', ...),
             n_jobs=10,
             param_grid={'colasample_bytree': [0.8, 0.9, 1],
                         'learning_rate': [0.1, 0.01, 0.001],
                         'max_depth': [3, 4, 5],
                         'n_estimators': [100, 200, 300],
                         'subsample': [0.8, 0.9, 1]},
             scoring='f1', verbose=1)

In [28]:
print("Best hyperparameters:\n", grid_search.best_params_)

Best hyperparameters:
 {'colasample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300, 'subsample': 0.9}


In [29]:
#obtaining best model from grid search
best_model = grid_search.best_estimator_

In [30]:
# Save the XGBoost model to a file
best_model.save_model('./Models/xgboost_best_model.model')

/home/arnold/anaconda3/envs/ml/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [08:45:01] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/arnold/anaconda3/envs/ml/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [08:45:01] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [31]:
#prediction using best model
y_pred_tuned = best_model.predict(X_test)

/home/arnold/anaconda3/envs/ml/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [08:45:07] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


In [32]:
# Evaluate the model
print(classification_report(y_test, y_pred_tuned))

              precision    recall  f1-score   support

         0.0       0.93      0.92      0.93     45968
         1.0       0.32      0.36      0.34      4768

    accuracy                           0.87     50736
   macro avg       0.63      0.64      0.63     50736
weighted avg       0.88      0.87      0.87     50736



#### Creating a Xgb model with L1 and L2 regularization


In [48]:
params = {
    'tree_method': 'gpu_hist', #use GPU for training
    'predictor': 'gpu_predictor', # use GPU for inference
    'reg_alpha': 0.1,               # L1 regularization parameter
    'reg_lambda': 0.1,              # L2 regularization parameter
}

In [49]:
# XGBoots model with regularization
xgb_model_reg = xgb.XGBClassifier(**params, random_state=42)

In [50]:
# training the model
xgb_model_reg.fit(X_train_resampled, y_train_resampled)

/home/arnold/anaconda3/envs/ml/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [02:22:50] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/arnold/anaconda3/envs/ml/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [02:22:50] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, predictor='gpu_predictor', ...)

In [51]:
#Prediction of the model

y_pred_xgb_reg = xgb_model_reg.predict(X_test)

/home/arnold/anaconda3/envs/ml/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [02:23:57] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [52]:
# Evaluate the model
print(confusion_matrix(y_test, y_pred_xgb_reg))
print(classification_report(y_test, y_pred_xgb_reg))
print("Accuracy:", accuracy_score(y_test, y_pred_xgb_reg))

[[42900  3068]
 [ 3269  1499]]
              precision    recall  f1-score   support

         0.0       0.93      0.93      0.93     45968
         1.0       0.33      0.31      0.32      4768

    accuracy                           0.88     50736
   macro avg       0.63      0.62      0.63     50736
weighted avg       0.87      0.88      0.87     50736

Accuracy: 0.8750985493535163
